In [18]:
from pyds import MassFunction
import pandas as pd
from data_cleaning.data_cleaning import get_zimmer, get_stockwerk, get_heizung, get_kindergarten, get_schule, get_bahn, get_miete, get_nebenkosten, get_alter, get_lage, get_kaution, get_kueche, get_bad, get_mobliert, get_quadratmeter

In [19]:
file_path = '../../data/Wohnungen_1.csv'  # Gegebenen Datensatz einlesen
data = pd.read_csv(file_path, sep=';')

# Bewohnerkategorie bestimmen
bewohnerkategorie = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']
data['Bewohnerkategorie'] = data[bewohnerkategorie].apply(lambda row: 'Keine' if all(row == 'nein') else row.idxmax(), axis=1)

# Bereinigung der Daten
data = data[data['Bewohnerkategorie'] != 'Keine']

# Spalten, in denen "nein" durch 0 und "ja" durch 1 ersetzt werden sollen
columns_to_replace = ["Hausmeister", "Garage", "Aufzug", "Balkon", "Terrasse", "Kehrwoche"]

# Ersetze "nein" und "ja" in den angegebenen Spalten
data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})

data['Zimmerzahl'] = data['Zimmerzahl'].apply(get_zimmer)
data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)
data['Heizung'] = data['Heizung'].apply(get_heizung)
data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)
data['Schule'] = data['Schule'].apply(get_schule)
data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)
data['Miete'] = data['Miete'].apply(get_miete)
data['Nebenkosten'] = data['Nebenkosten'].apply(get_nebenkosten)
data['Alter'] = data['Alter'].apply(get_alter)
data['Lage'] = data['Lage'].apply(get_lage)
data['Kaution'] = data['Kaution'].apply(get_kaution)
data['Kueche'] = data['Kueche'].apply(get_kueche)
data['Bad'] = data['Bad'].apply(get_bad)
data['Moebliert'] = data['Moebliert'].apply(get_mobliert)
data['Quadratmeter'] = data['Quadratmeter'].apply(get_quadratmeter)

print(data.head())

    Zimmerzahl  Stockwerk  Heizung  Hausmeister  Kindergarten  Schule  S-Bahn  \
11           2          0        0            0             1       2       4   
13           2          0        5            0             2       2       3   
23           2          1        4            1             1       2       4   
27           0          0        1            0             1       0       0   
40           0          8        5            0             0       2       2   

    Garage  Miete  Nebenkosten  ...  Kehrwoche  Moebliert  Quadratmeter  \
11       0    726          276  ...          0          0           116   
13       1    726          226  ...          0          0           106   
23       0    776          276  ...          0          0           116   
27       0    226           50  ...          0          0            25   
40       0    226           50  ...          0          0            36   

    Studierende  Kleinfamilie  DINK  Alleinerziehende  Expatri

/tmp/ipykernel_8781/1129585513.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})


In [20]:
omega = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']

In [21]:
# Berechne Basismaße für Garage
basismaße_garage = {}

for kategorie in omega:
    total = len(data)
    count_hypothesis = len(data[(data['Garage'] == 1) & (data['Bewohnerkategorie'] == kategorie)])
    basismaße_garage[frozenset([kategorie])] = count_hypothesis / total

# Unsicherheit hinzufügen
basismaße_garage[frozenset()] = 1 - sum(basismaße_garage.values())

# Erstelle die MassFunction für Garage
mass_garage = MassFunction(basismaße_garage)
print("Basismaße für Garage:", mass_garage)


Basismaße für Garage: {set():0.7164179104477613; {'Studierende'}:0.208955223880597; {'Kleinfamilie'}:0.07462686567164178; {'DINK'}:0.0; {'Alleinerziehende'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}


In [26]:
# Berechne Basismaße für Aufzug
basismaße_aufzug = {}

for kategorie in omega:
    total = len(data)
    count_hypothesis = len(data[(data['Aufzug'] == 1) & (data['Bewohnerkategorie'] == kategorie)])
    basismaße_aufzug[frozenset([kategorie])] = count_hypothesis / total

# Unsicherheit hinzufügen
basismaße_aufzug[frozenset()] = 1 - sum(basismaße_aufzug.values())

# Erstelle die MassFunction für Garage
mass_aufzug = MassFunction(basismaße_aufzug)
print("Basismaße für Aufzug:", mass_aufzug)

Basismaße für Aufzug: {set():0.417910447761194; {'Studierende'}:0.3880597014925373; {'Kleinfamilie'}:0.19402985074626866; {'DINK'}:0.0; {'Alleinerziehende'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}


In [30]:
# Kombiniere Basismaße (z. B. Garage und Zimmerkategorie)
combined_mass = mass_garage & mass_aufzug
print("Kombinierte Evidenz:", combined_mass)

# Wahrscheinlichste Hypothese auswählen
most_likely = combined_mass.max_bel()
print("Am wahrscheinlichsten ist:", most_likely)


Kombinierte Evidenz: {{'Studierende'}:0.8484848484848485; {'Kleinfamilie'}:0.15151515151515152; {'DINK'}:0.0; {'Alleinerziehende'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Am wahrscheinlichsten ist: frozenset({'Studierende'})
